In [ ]:
from Cocordiais import CocordiaisUtils as cocordiais

import datasets
import json
import os
import pandas as pd

from huggingface_hub import DatasetCard, DatasetCardData

In [ ]:
DATA_PATH = "./data"

DATA_INFO = [
  {
    "name": "cordiais",
    "license_id": 1,
    "date": "2023-06-10 00:00:00",
    "include_pct": 0.5,
    "source_path": os.path.join(DATA_PATH, "cordiais-source"),
    "json_path": os.path.join(DATA_PATH, "cordiais.json")
  },
  {
    "name": "metfaces",
    "license_id": 1,
    "date": "2020-06-10 00:00:00",
    "include_pct": 1.0,
    "source_path": os.path.join(DATA_PATH, "metfaces-source"),
    "json_path": os.path.join(DATA_PATH, "metfaces.json")
  }
]

for ds in DATA_INFO:
  file_list = sorted([f.replace(".jpg", "") for f in os.listdir(ds["source_path"]) if f.endswith(".jpg")])
  include_length = int(ds["include_pct"] * len(file_list))
  ds["source_list"] = file_list[:include_length]

ALL_IMGS = [img for subl in [ds["source_list"] for ds in DATA_INFO] for img in subl]
IMG2ID = {img:id for id,img in enumerate(ALL_IMGS)}


### Create HF Dataset

In [ ]:
slug2obj = {}
object_count = 0

for ds in DATA_INFO:
  with open(ds["json_path"]) as json_file_read:
    data_json = json.load(json_file_read)

    for object_info in data_json:
      object_slug = object_info["source_image"]

      if object_slug in ds["source_list"]:
        if object_slug not in slug2obj:
          slug2obj[object_slug] = {
            "image_id": IMG2ID[object_slug],
            "image": os.path.join(ds["source_path"], "%s.jpg" % object_slug),
            "image_filename": "%s.jpg" % object_slug,
            "width": object_info["source_image_w"],
            "height": object_info["source_image_h"],
            "license_id": ds["license_id"],
            "date_captured": ds["date"],
            "objects": []
          }

        slug2obj[object_slug]["objects"].append({
          "bbox_id": object_count,
          "area": object_info["face_rect_xywh"][2] * object_info["face_rect_xywh"][3],
          "bbox": object_info["face_rect_xywh"],
          "category": object_info["gender"],
          "super_category": cocordiais.LABEL2SUPERLABEL[object_info["gender"]],
          "is_crowd": False
        })
        object_count += 1

records = json.loads(json.dumps(list(slug2obj.values())))


In [ ]:
for r in records:
  r["objects"] = pd.DataFrame(r["objects"]).to_dict('list')

df_dict = pd.DataFrame(records).to_dict('list')


In [ ]:
CORDIAIS_FEATURES = datasets.Features({
  "image_id": datasets.Value("int64"),
  "image": datasets.Image(decode=True),
  "image_filename": datasets.Value("string"),
  "width": datasets.Value("int64"),
  "height": datasets.Value("int64"),
  "license_id": datasets.Value("int64"),
  "date_captured": datasets.Value("string"),
  "objects": datasets.Sequence(feature={
    "bbox_id": datasets.Value("int64"),
    "category": datasets.ClassLabel(names=list(cocordiais.LABEL2ID.keys())),
    "bbox": datasets.Sequence(feature=datasets.Value("int64"), length=4),
    "super_category": datasets.ClassLabel(names=list(set(cocordiais.COCORDIAIS_SUPERLABELS))),
    "area": datasets.Value("int64"),
    "is_crowd": datasets.Value("bool")
  })
})

In [ ]:
hf_dataset = datasets.Dataset.from_dict(df_dict, features=CORDIAIS_FEATURES)
hf_dataset.info.description = cocordiais.COCORDIAIS_DATASET_INFO["info"]["description"]
hf_dataset.info.version = cocordiais.COCORDIAIS_DATASET_INFO["info"]["version"]
hf_dataset.info.license = cocordiais.COCORDIAIS_DATASET_INFO["licenses"][0]["name"]

In [ ]:
hf_dataset.push_to_hub("thiagohersan/cordiais-faces", private=True)